
## Autograded Notebook (Canvas & CodeGrade)

This notebook will be automatically graded. It is designed to test your answers and award points for the correct answers. Following the instructions for each Task carefully.
Instructions

- **Download** this notebook as you would any other ipynb file 
- **Upload** to Google Colab or work locally (if you have that set-up)
- **Delete** `raise NotImplementedError()`

- **Write** your code in the `# YOUR CODE HERE` space


- **Execute** the Test cells that contain assert statements - these help you check your work (others contain hidden tests that will be checked when you submit through Canvas)

- **Save** your notebook when you are finished
- **Download** as a ipynb file (if working in Colab)
- **Upload** your complete notebook to Canvas (there will be additional instructions in Slack and/or Canvas)



# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more manageable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
Successfully complete all these objectives to earn full credit. 

**Successful completion is defined as passing all the unit tests in each objective.**  

Each unit test that you pass is 1 point. 

There are 5 total possible points in this sprint challenge. 


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernel

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu. 

3) Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section). 
____



### Import Data

In [66]:
import pandas as pd
pd.set_option("display.max_columns", None)
# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
# YOUR CODE HERE
df = pd.read_json(data_url, lines=True)

In [45]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [47]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.
import re
import spacy
nlp = spacy.load('en_core_web_sm')

In [46]:
# STOP_WORDS = nlp.Defaults.stop_words.union(['s', 'year', 'american'])
def tokenize(doc):
    """
    Takes a doc and returns a list of tokens in the form of lemmas
    Non-alphabet, stop words, punctation, and pronoun are filtered out.
    """
    # use regex to strip out multi white space and non alphabet
    non_alpha = '[^a-zA-Z]'
    multi_white_spaces = "[ ]{2,}"
    doc = re.sub(non_alpha, ' ', doc)
    doc = re.sub(multi_white_spaces, " ", doc)
    
    # use nlp
    doc = nlp(doc)
    
    return [token.strip() for token in doc if (token.is_stop != True) and 
            (token.is_punct != True) and (token.pos_ != 'PRON')]

In [48]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
2. Write a fake review and query for the 10 most similar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, use `NearestNeighbors` model for this. 

In [15]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
# Create a vector representation of the reviews 
tfidf_vect = TfidfVectorizer(tokenizer=tokenize)

# Name that doc-term matrix "dtm"
dtm = tfidf_vect.fit_transform(df['text'])

# View Feature Matrix as DataFrame
dtm = pd.DataFrame(data=dtm.toarray(), columns=tfidf_vect.get_feature_names())

CPU times: user 2min, sys: 242 ms, total: 2min
Wall time: 2min


,,aa,aaa,aaaahhhs,aaasssk,aab,aamco,aand,aaron,aback,...,zuni,zupas,zur,zuzana,zuzu,zxkxko,zyr,zyrtec,zzaplon,zzzzzzzzz
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.156418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors

# YOUR CODE HERE
nn = NearestNeighbors(n_neighbors=10).fit(dtm)

#### "instructor confirmed that unit test needs to be updated"

In [34]:
nn.__module__

'sklearn.neighbors._unsupervised'

In [38]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors.unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

AssertionError:  nn is not a NearestNeighbors instance.

In [50]:
# Create a fake review and find the 10 most similar reviews
fake_review = 'Fake. This is the worst company I have ever seen. I could not believe what they did to me. I already called their office number to talk to their manager. They never answer the phone'
tfidf_vect = TfidfVectorizer(tokenizer=tokenize)
# YOUR CODE HERE
df_new = df['text'].copy()
df_new.loc[len(df_new.index)] = fake_review
dtm_new = tfidf_vect.fit_transform(df_new)
# View Feature Matrix as DataFrame
dtm_new = pd.DataFrame(data=dtm_new.toarray(), columns=tfidf_vect.get_feature_names())


In [56]:
# Fit into NearestNeighbors
nn = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(dtm_new)
doc = [dtm_new.iloc[-1].values]

# Query Using kneighbors 
neigh_dist, neigh_index = nn.kneighbors(doc)

In [67]:
df_new.loc[df_new.index.isin(neigh_index[0])]

0        BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...
2943     Well, from the outside it looks like a pretty ...
3180     This Walmart has the rudest of employees I hav...
4406     Probably the worst HVAC service I have used. A...
4491     This is a update to my earlier review. The mec...
5956     Yesterday my two friends and I were at Madison...
6019     I overall liked the atmosphere of this locatio...
8470     if could leave a 0 star i would i was on hold ...
9587     Other than the pricing, this company is awful....
10000    Fake. This is the worst company I have ever se...
Name: text, dtype: object

#### All the text are negative review of the companies. There is a pattern that using these word: worst, rudest, fake.

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels). 
    - Use that Pipeline to predict a star rating for your fake review from Part 2. 



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`. 
    - Include 2 possible values for each parameter
    - **Use `n_jobs` = 1** 
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [70]:
df.stars.value_counts()

5    4462
4    2185
1    1496
3    1098
2     759
Name: stars, dtype: int64

In [69]:
X = df['text']
Y = df['stars']

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


# Instantiate model
vect = TfidfVectorizer(tokenizer=tokenize)
clf = KNeighborsClassifier()
pipe = Pipeline([('vect', vect),
                 ('clf', clf)])

# create a hyper-parameter dict
params = {
    "vect__min_df": [.5, .01],
    "clf__n_estimators": [50, 100]  
}

# Name the gridsearch instance "gs"
gs = GridSearchCV(pipe, 
                  params, 
                  n_jobs=1, 
                  cv=3, 
                  verbose=1)

# run the gridsearch
gs.fit(X, Y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x7f2c0b2a49d0>)),
                                       ('clf', KNeighborsClassifier())]),
             n_jobs=1,
             param_grid={'clf__n_neighbors': [15, 10],
                         'vect__min_df': [0.05, 0.01]},
             verbose=1)

In [80]:
# Visible Testing
prediction = gs.predict(["I wish dogs knew how to speak English."])[0]
assert prediction in df.stars.values, 'You gs object should be able to accept raw text within a list. Did you include a vectorizer in your pipeline?'

/home/minh14496/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-1TUUJhOU/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this: 

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.** 

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz). 

In [73]:
from gensim import corpora
# Due to limited computationalresources on CodeGrader, use the non-multicore version of LDA 
from gensim.models.ldamodel import LdaModel
import gensim
import re

/home/minh14496/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-1TUUJhOU/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### 1. Estimate a LDA topic model of the review tex

In [74]:
# Remember to read the LDA docs for more information on the various class attirbutes and methods available to you
# in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

# don't change this value 
num_topics = 5

# use tokenize function you created earlier to create tokens 
df['tokens'] = df['text'].apply(tokenize)
# create a id2word object (hint: use corpora.Dictionary)
id2word = corpora.Dictionary(df['tokens'] )
# create a corpus object (hint: id2word.doc2bow)
corpus = [id2word.doc2bow(text) for text in df['tokens']]
# instantiate an lda model
lda = LdaModel(corpus, num_topics)

#### Testing

In [75]:
# Visible Testing
assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

#### 2. Create 1-2 visualizations of the results

In [79]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pyLDAvis
# import pyLDAvis.gensim_models
# Use pyLDAvis (or a ploting tool of your choice) to visualize your results 
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim_models.prepare(lda, corpus, id2word)
# vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.134367  0.032756       1        1  33.773605
3      0.019387 -0.051339       2        1  25.880258
4      0.032215 -0.063037       3        1  23.195110
0     -0.010041  0.001434       4        1  12.608945
1      0.092806  0.080186       5        1   4.542082, topic_info=      Term         Freq        Total Category  logprob  loglift
199  great  4040.000000  4040.000000  Default  30.0000  30.0000
138   food  4580.000000  4580.000000  Default  29.0000  29.0000
43    good  6177.000000  6177.000000  Default  28.0000  28.0000
720  sushi   408.000000   408.000000  Default  27.0000  27.0000
125  place  5241.000000  5241.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
40    come    79.499674  3916.912977   Topic5  -5.6650  -0.8055
17    like    77.801394  3775.652577   Topic5  -5.6866  -0.7904
678   love    71.714606  2101.265261   Topic5  -5.7681  -0.2858
368  order    70.655524  3438.615423   Topic5  -5.7830  -0.7932
383  staff    66.992312  1521.280237   Topic5  -5.8362  -0.0309

[483 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
7052       4  0.966998    Alex
14472      1  0.977959  Amazon
1950       3  0.071287     Ami
1950       5  0.926737     Ami
6326       2  0.912650  Bobbie
...      ...       ...     ...
112        2  0.116351    year
112        3  0.085257    year
112        4  0.200606    year
112        5  0.028085    year
12598      2  0.982223     yuk

[1208 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 1, 2])

#### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

It seems like topic 5 is about japanese food/ restaurant because they have sushi and fish. Topic 3 is about mexican food/restaurant. Topic two is about italian food/restaurant. These two topics 2,3 is nearly the same because they have more similar words: cheese, chicken, salad. Topic 4 is fastfood restaurant. It is close to these two topics above but because they have some similar word: chicken, salad. Topic 1 is about other companies: hotel, doctor, pool, anything not a restaurant.